In [1]:
#Import a variety of things
# To create TS geometry guesses

import numpy as np
import logging
FORMAT = "%(filename)s:%(lineno)d %(funcName)s %(levelname)s %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO)

import rdkit, rdkit.Chem.rdDistGeom, rdkit.DistanceGeometry

from rdkit import Chem

from rdkit.Chem import AllChem
from rdkit.Chem.Pharm3D import EmbedLib

import ase

import rmgpy
from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy.reaction import Reaction, ReactionError
from rmgpy.kinetics import PDepArrhenius, PDepKineticsModel
from rmgpy.data.rmg import RMGDatabase

import os
import sys
from autotst.reaction import AutoTST_Reaction, AutoTST_TS
from autotst.molecule import AutoTST_Molecule
from autotst.geometry import Bond, Angle, Torsion
from ase.io.gaussian import read_gaussian_out

# To perform TS search
from ase.calculators.gaussian import Gaussian
from autotst.calculators.gaussian import AutoTST_Gaussian
from autotst.calculators.vibrational_analysis import Vibrational_Analysis, percent_change
from autotst.calculators.cantherm import AutoTST_CanTherm


reaction_string = "CO+[O]O_C[O]+OO"

logging.info("Running the following reaction: {}".format(reaction_string))

scratch = "/gss_gpfs_scratch/harms.n/"

reaction_string = reaction_string.strip("\n")
test_reaction = AutoTST_Reaction(reaction_string, "H_Abstraction")

test_reaction.ts.view_ts()
### Performing the partial optimizations
tst_calculators = AutoTST_Gaussian(test_reaction, scratch=scratch, save_directory=".")

kinetics = tst_calculators.read_kinetics_file()

if kinetics:
    logging.info("We have previously loaded kinetics:")
    logging.info("{0!r}".format(kinetics['reaction']))

else:
    gaussian_results = tst_calculators.run_all()
    if gaussian_results:
        ### Running CanTherm ###
        cantherm = AutoTST_CanTherm(tst_calculators.reaction, scratch=scratch, output_directory=scratch)
        cantherm.write_files()
        cantherm.run()
        cantherm.set_reactants_and_products()

        ### Printing Results ###
        logging.info("The kinetics of intrest are as follows:")
        logging.info("{0!r}".format(cantherm.kinetics_job.reaction))

        tst_calculators.save_kinetics(tst_calculators.method, cantherm.kinetics_job.reaction)

    else:
        logging.info("Failed gaussian... :(")



<ipython-input-1-5dcade385f8f>:42 <module> INFO Running the following reaction: CO+[O]O_C[O]+OO
reaction.py:191 load_databases INFO Loading RMG database from '/home/harms.n/Code/RMG-database/input'
thermo.py:839 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /home/harms.n/Code/RMG-database/input/thermo/libraries...
thermo.py:839 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /home/harms.n/Code/RMG-database/input/thermo/libraries...
thermo.py:839 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /home/harms.n/Code/RMG-database/input/thermo/libraries...
thermo.py:856 loadGroups INFO Loading thermodynamics group database from /home/harms.n/Code/RMG-database/input/thermo/groups...
transport.py:294 loadGroups INFO Loading transport group database from /home/harms.n/Code/RMG-database/input/transport/groups...
statmech.py:526 loadGroups INFO Loading frequencies group database from /home/harms.n/Co

CanTherm execution initiated at Thu Jul 19 13:39:34 2018

###############################################################
#                                                             #
#                          CanTherm                           #
#                                                             #
#   Version: 2.1.9                                            #
#   Authors: RMG Developers (rmg_dev@mit.edu)                 #
#   P.I.s:   William H. Green (whgreen@mit.edu)               #
#            Richard H. West (r.west@neu.edu)                 #
#   Website: http://reactionmechanismgenerator.github.io/     #
#                                                             #
###############################################################

Loading species CO...
Loading species [O]O...
Loading species C[O]...
Loading species OO...
Loading transition state TS...
Loading reaction CO+[O]O_C[O]+OO...


Loading statistical mechanics parameters for CO...
Saving statistical mechan

ValueError: One or both of the barrier heights of -1.32472 and 72.7617 kJ/mol encountered in Eckart method are invalid.

In [ ]:
### Performing the partial optimizations
tst_calculators = AutoTST_Gaussian(test_reaction, scratch=scratch, save_directory=".")

kinetics = tst_calculators.read_kinetics_file()

if kinetics:
    logging.info("We have previously loaded kinetics:")
    logging.info("{0!r}".format(kinetics['reaction']))

else:
    gaussian_results = tst_calculators.run_all()
    if gaussian_results:
        ### Running CanTherm ###
        cantherm = AutoTST_CanTherm(tst_calculators.reaction, scratch=scratch, output_directory=scratch)
        cantherm.write_files()
        cantherm.run()
        cantherm.set_reactants_and_products()

        ### Printing Results ###
        logging.info("The kinetics of intrest are as follows:")
        logging.info("{0!r}".format(cantherm.kinetics_job.reaction))

        tst_calculators.save_kinetics(tst_calculators.method, cantherm.kinetics_job.reaction)

    else:
        logging.info("Failed gaussian... :(")